# 「單一」股票最佳擇時(Timing)因子選擇

In [1]:
"""
因子優化模組使用範例
"""
from alanq.optimization import ParameterSpace, FactorOptimizer
from alanq.factors.timing import BreakoutBuyFactor, BreakdownSellFactor, AtrStopSellFactor
from alanq.data import StockDataManager

def example_optimization():
    """範例：單股票因子優化"""
    
    # 1. 準備資料
    print("正在下載資料...")
    data_manager = StockDataManager(["TSLA"], start_date="2020-01-01")
    df = data_manager.get_kl_pd("TSLA")
    
    # 2. 定義參數空間
    print("定義參數空間...")
    param_space = ParameterSpace()
    
    # 添加買入因子
    param_space.add_buy_factor(
        BreakoutBuyFactor,
        {'xd': [20, 30, 40, 50, 60], 'skip_days': [10, 15, 20]}
    )
    
    # 添加賣出因子
    param_space.add_sell_factor(
        BreakdownSellFactor,
        {'xd': [10, 15, 20, 25]}
    )
    
    param_space.add_sell_factor(
        AtrStopSellFactor,
        {'atr_n': [1.0, 1.5, 2.0, 2.5]}
    )
    
    # 查看總組合數
    total_combos = param_space.get_total_combinations()
    print(f"總組合數: {total_combos}")
    
    # 3. 執行優化
    print("\n開始優化...")
    # 定義自定義權重
    custom_weights = {
        '策略_總報酬率': 1,     
        '策略_Sharpe': 1,       
        '策略_最大回撤': 1,     
        '勝率': 1,              
        '盈虧比': 1,             
        # 其他指標權重為 0（不考慮）
    }

    # 使用自定義權重進行優化
    optimizer = FactorOptimizer(
        df=df,
        parameter_space=param_space,
        metric_weights=custom_weights,  # 傳入自定義權重
        initial_capital=1_000_000,
        show_progress=True
    )
    # optimizer = FactorOptimizer(
    #     df=df,
    #     parameter_space=param_space,
    #     initial_capital=1_000_000,
    #     show_progress=True
    # )
    
    best_config, results_df = optimizer.optimize()
    
    # 4. 查看結果
    if best_config:
        print(f"\n{'='*60}")
        print("最佳配置:")
        print(f"{'='*60}")
        print(f"買入因子: {best_config['buy_factors']}")
        print(f"賣出因子: {best_config['sell_factors']}")
        print(f"總得分: {best_config['總得分']:.2f}")
        
        print(f"\n{'='*60}")
        print("前 10 名結果:")
        print(f"{'='*60}")
        top_results = optimizer.get_top_n(10)
        display_cols = ['組合編號', '總得分', '策略_總報酬率', '策略_Sharpe', 
                       '策略_最大回撤', '勝率', '盈虧比']
        available_cols = [col for col in display_cols if col in top_results.columns]
        print(top_results[available_cols].to_string())
    else:
        print("優化失敗，沒有找到有效結果")
    
    return best_config, results_df

if __name__ == "__main__":
    best_config, results_df = example_optimization()

正在下載資料...
正在使用 yfinance 下載 1 檔股票資料...
------------------------------
已成功下載 1 檔股票資料
  - TSLA: 1482 筆資料，日期範圍 2020-01-02 至 2025-11-21
------------------------------
定義參數空間...
總組合數: 240

開始優化...
總共需要測試 240 種組合...


收集指標數據: 100%|██████████| 240/240 [00:24<00:00,  9.82it/s]



計算統計分佈並標準化得分...
使用的歸一化權重（總和為 1.0）:
  - 策略_總報酬率: 0.2000
  - 策略_Sharpe: 0.2000
  - 策略_最大回撤: 0.2000
  - 勝率: 0.2000
  - 盈虧比: 0.2000

最佳配置:
買入因子: [{'class': <class 'alanq.factors.timing.breakout_factor.BreakoutBuyFactor'>, 'xd': 20, 'skip_days': 10}]
賣出因子: [{'class': <class 'alanq.factors.timing.breakout_factor.BreakdownSellFactor'>, 'xd': 10}, {'class': <class 'alanq.factors.timing.atr_stop_sell_factor.AtrStopSellFactor'>, 'atr_n': 1.0}]
總得分: 58.97

前 10 名結果:
          總得分    策略_總報酬率  策略_Sharpe   策略_最大回撤
0   58.974214  12.877063   1.102709 -0.393893
1   58.974214  12.877063   1.102709 -0.393893
2   58.974214  12.877063   1.102709 -0.393893
3   58.974214  12.877063   1.102709 -0.393893
16  58.974214  12.877063   1.102709 -0.393893
17  58.974214  12.877063   1.102709 -0.393893
18  58.974214  12.877063   1.102709 -0.393893
19  58.974214  12.877063   1.102709 -0.393893
32  58.974214  12.877063   1.102709 -0.393893
33  58.974214  12.877063   1.102709 -0.393893


In [2]:
results_df

,總得分,買入因子,賣出因子,策略_總報酬率_得分,策略_年化報酬率_得分,策略_Sharpe_得分,策略_最大回撤_得分,策略_年化波動率_得分,勝率_得分,盈虧比_得分,...,profit_loss_ratio,avg_profit,avg_loss,net_profit,max_single_profit,max_single_loss,avg_holding_days,avg_return,max_consecutive_wins,max_consecutive_losses
0,58.974214,[{'class': <class 'alanq.factors.timing.breako...,[{'class': <class 'alanq.factors.timing.breako...,20.000000,0,20.000000,4.934675,0,6.218202,7.821337,...,2.893340,1.267237e+06,437984.189758,1.287678e+07,4.139933e+06,-1.113653e+06,27.896552,0.117906,4,4
1,58.974214,[{'class': <class 'alanq.factors.timing.breako...,[{'class': <class 'alanq.factors.timing.breako...,20.000000,0,20.000000,4.934675,0,6.218202,7.821337,...,2.893340,1.267237e+06,437984.189758,1.287678e+07,4.139933e+06,-1.113653e+06,27.896552,0.117906,4,4
2,58.974214,[{'class': <class 'alanq.factors.timing.breako...,[{'class': <class 'alanq.factors.timing.breako...,20.000000,0,20.000000,4.934675,0,6.218202,7.821337,...,2.893340,1.267237e+06,437984.189758,1.287678e+07,4.139933e+06,-1.113653e+06,27.896552,0.117906,4,4
3,58.974214,[{'class': <class 'alanq.factors.timing.breako...,[{'class': <class 'alanq.factors.timing.breako...,20.000000,0,20.000000,4.934675,0,6.218202,7.821337,...,2.893340,1.267237e+06,437984.189758,1.287678e+07,4.139933e+06,-1.113653e+06,27.896552,0.117906,4,4
4,44.475218,[{'class': <class 'alanq.factors.timing.breako...,[{'class': <class 'alanq.factors.timing.breako...,12.500439,0,14.951714,6.930613,0,6.314511,3.777941,...,2.088549,1.057462e+06,506313.986393,8.222383e+06,2.305038e+06,-8.197825e+05,36.259259,0.122823,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,42.960132,[{'class': <class 'alanq.factors.timing.breako...,[{'class': <class 'alanq.factors.timing.breako...,5.240043,0,11.210363,9.023386,0,17.486339,0.000000,...,1.336595,7.421636e+05,555264.420265,3.716251e+06,1.513443e+06,-9.460377e+05,61.833333,0.171668,4,3
236,56.402378,[{'class': <class 'alanq.factors.timing.breako...,[{'class': <class 'alanq.factors.timing.breako...,8.204530,0,13.511047,13.207720,0,20.000000,1.479082,...,1.630989,1.076630e+06,660109.067139,5.556086e+06,1.644835e+06,-1.234579e+06,84.700000,0.270484,4,3
237,56.402378,[{'class': <class 'alanq.factors.timing.breako...,[{'class': <class 'alanq.factors.timing.breako...,8.204530,0,13.511047,13.207720,0,20.000000,1.479082,...,1.630989,1.076630e+06,660109.067139,5.556086e+06,1.644835e+06,-1.234579e+06,84.700000,0.270484,4,3
238,56.402378,[{'class': <class 'alanq.factors.timing.breako...,[{'class': <class 'alanq.factors.timing.breako...,8.204530,0,13.511047,13.207720,0,20.000000,1.479082,...,1.630989,1.076630e+06,660109.067139,5.556086e+06,1.644835e+06,-1.234579e+06,84.700000,0.270484,4,3


# 多股投資組合最佳擇時(Timing)因子選擇
* 一般先用擇股(Selection)模組選出股票，再進行擇時

In [34]:

from alanq.optimization import ParameterSpace, MultiStockFactorOptimizer
from alanq.factors.timing import BreakoutBuyFactor, BreakdownSellFactor
from alanq.data import StockDataManager

# 1. 準備多股票資料
data_manager = StockDataManager(["AAPL", "TSLA", "MSFT"], start_date="2020-01-01")
stock_data = {symbol: data_manager.get_kl_pd(symbol) for symbol in ["AAPL", "TSLA", "MSFT"]}

# 2. 定義參數空間
param_space = ParameterSpace()
param_space.add_buy_factor(BreakoutBuyFactor, {'xd': [20, 40, 60]})
param_space.add_sell_factor(BreakdownSellFactor, {'xd': [10, 20, 25]})

custom_weights = {
        '策略_總報酬率': 1,     
        '策略_Sharpe': 1,       
        '策略_最大回撤': 3,            
        # 其他指標權重為 0（不考慮）
    }

# 3. 執行優化
optimizer = MultiStockFactorOptimizer(
        stock_data=stock_data,              # 必要：股票資料字典
        parameter_space=param_space,          # 必要：參數空間定義
        initial_capital=1_000_000,           # 可選：初始資金（預設 1,000,000）
        slippage_factors=None,   # 可選：滑價因子列表（預設 None）
        position_manager=None,    # 可選：倉位管理器（預設 None）
        show_progress=True,                  # 可選：顯示進度條（預設 True）
        n_jobs=-1,                          # 可選：並行進程數，-1=所有核心（預設 1）
        metric_weights=custom_weights,       # 可選：指標權重字典（預設 None=等權重）
        enable_full_rate_factor=False
    )

best_config, results_df = optimizer.optimize()

# 4. 查看結果
print(f"最佳配置: {best_config}")
print(f"前 10 名: {optimizer.get_top_n(10)}")

正在使用 yfinance 下載 3 檔股票資料...
------------------------------
已成功下載 3 檔股票資料
  - AAPL: 1482 筆資料，日期範圍 2020-01-02 至 2025-11-21
  - TSLA: 1482 筆資料，日期範圍 2020-01-02 至 2025-11-21
  - MSFT: 1481 筆資料，日期範圍 2020-01-02 至 2025-11-20
------------------------------
總共需要測試 9 種組合...
股票數量: 3
股票代號: ['AAPL', 'TSLA', 'MSFT']
使用 8 個進程進行並行處理...


收集指標數據: 100%|██████████| 9/9 [00:05<00:00,  1.72it/s]


計算統計分佈並標準化得分...
使用的歸一化權重（總和為 1.0）:
  - 策略_總報酬率: 0.2000
  - 策略_Sharpe: 0.2000
  - 策略_最大回撤: 0.6000
最佳配置: {'buy_factors': "[{'class': <class 'alanq.factors.timing.breakout_factor.BreakoutBuyFactor'>, 'xd': 20}]", 'sell_factors': "[{'class': <class 'alanq.factors.timing.breakout_factor.BreakdownSellFactor'>, 'xd': 25}]", '總得分': np.float64(68.36944447213956)}
前 10 名:          總得分                                               買入因子  \
2  68.369444  [{'class': <class 'alanq.factors.timing.breako...   
5  66.062369  [{'class': <class 'alanq.factors.timing.breako...   
8  64.751464  [{'class': <class 'alanq.factors.timing.breako...   
4  48.336156  [{'class': <class 'alanq.factors.timing.breako...   
1  47.041291  [{'class': <class 'alanq.factors.timing.breako...   
3  42.587140  [{'class': <class 'alanq.factors.timing.breako...   
6  17.813842  [{'class': <class 'alanq.factors.timing.breako...   
0  14.082545  [{'class': <class 'alanq.factors.timing.breako...   
7  11.549319  [{'class': <class

In [35]:
best_config

{'buy_factors': "[{'class': <class 'alanq.factors.timing.breakout_factor.BreakoutBuyFactor'>, 'xd': 20}]",
 'sell_factors': "[{'class': <class 'alanq.factors.timing.breakout_factor.BreakdownSellFactor'>, 'xd': 25}]",
 '總得分': np.float64(68.36944447213956)}

# MultiStockFactorOptimizer 的績效指標修改
* 將原先 custom_weights 的 dict[str, float] 結構轉換為績效指標結構：
```
{
    "class": BaseBasicMetric 或 BaseDetailMetric 的子類,
    "weight": float
}
```

In [36]:
from alanq.performance import (
    TotalReturnMetric,
    SharpeMetric,
    MaxDrawdownMetric,
    WinRateMetric,
    ProfitLossRatioMetric,
    NetProfitMetric
)

# =========================================================
# 更新範例 ：多股票優化（使用指標類別權重）
# =========================================================
def example_multi_stock_optimization():
    """多股票優化，使用指標類別權重"""
    
    # 1. 準備多股票資料
    data_manager = StockDataManager(["AAPL", "TSLA", "MSFT"], start_date="2020-01-01")
    stock_data = {
        symbol: data_manager.get_kl_pd(symbol) 
        for symbol in ["AAPL", "TSLA", "MSFT"]
    }
    
    # 2. 定義參數空間
    param_space = ParameterSpace()
    param_space.add_buy_factor(BreakoutBuyFactor, {'xd': [20, 40, 60]})
    param_space.add_sell_factor(BreakdownSellFactor, {'xd': [10, 20, 25]})
    
    # 3. 使用指標類別定義權重
    metric_weights = [
        {"class": TotalReturnMetric, "weight": 1},
        {"class": SharpeMetric, "weight": 1},        
        {"class": MaxDrawdownMetric, "weight": 3},
        # {"class": WinRateMetric, "weight": 1},
        # {"class": ProfitLossRatioMetric, "weight": 1},  
        # {"class": NetProfitMetric, "weight": 1},        
    ]
    
    # 4. 執行優化
    optimizer = MultiStockFactorOptimizer(
        stock_data=stock_data,
        parameter_space=param_space,
        initial_capital=1_000_000,
        metric_weights=metric_weights,  # 傳入指標類別列表
        show_progress=True,
        n_jobs=-1,  # 使用所有 CPU 核心
        enable_full_rate_factor=False
    )
    
    best_config, results_df = optimizer.optimize()
    
    return best_config, results_df


# =========================================================
# 範例 5：查看指標類別的屬性
# =========================================================
def example_view_metric_attributes():
    """查看指標類別的屬性"""
    
    print("指標類別屬性範例：")
    print("=" * 60)
    
    # 創建指標實例以查看屬性
    metrics_to_check = [
        TotalReturnMetric(),
        SharpeMetric(),
        MaxDrawdownMetric(),
        WinRateMetric(),
    ]
    
    for metric in metrics_to_check:
        print(f"\n{metric.__class__.__name__}")
        print(f"  指標名稱 (metric_name): {metric.metric_name}")
        print(f"  該指標是否越大越好 (higher_is_better): {metric.higher_is_better}")
        print(f"  目標值 (target) -> 用於單股backtester，已棄用: {metric.target}")
        print(f"  描述 (description): {metric.description}")



# 查看指標類別屬性
# example_view_metric_attributes()
best_config_performence_opt, results_df_performence_opt = example_multi_stock_optimization()

正在使用 yfinance 下載 3 檔股票資料...
------------------------------
已成功下載 3 檔股票資料
  - AAPL: 1482 筆資料，日期範圍 2020-01-02 至 2025-11-21
  - TSLA: 1482 筆資料，日期範圍 2020-01-02 至 2025-11-21
  - MSFT: 1482 筆資料，日期範圍 2020-01-02 至 2025-11-21
------------------------------
總共需要測試 9 種組合...
股票數量: 3
股票代號: ['AAPL', 'TSLA', 'MSFT']
使用 8 個進程進行並行處理...


收集指標數據: 100%|██████████| 9/9 [00:05<00:00,  1.75it/s]


計算統計分佈並標準化得分...
使用的歸一化權重（總和為 1.0）:
  - 策略_總報酬率: 0.2000
  - 策略_Sharpe: 0.2000
  - 策略_最大回撤: 0.6000


In [41]:
# TEST: 看看相同參數不同使用方法是否得到相同的最佳參數
# best_config：使用dict[str, float] 權重
# best_config_performence_opt：使用指標類別權重
# 結果應該相同
print(best_config_performence_opt['buy_factors'] == best_config['buy_factors'])
print(best_config_performence_opt['sell_factors'] == best_config['sell_factors'])


True
True
